# Selector Group Chat

{py:class}`~autogen_agentchat.teams.SelectorGroupChat` implements a team where participants take turns broadcasting messages to all other participants,
with the next speaker selected by a generative model (e.g., an LLM) based on the shared context. 
This enables dynamic and context-aware multi-agent collaboration.

{py:class}`~autogen_agentchat.teams.SelectorGroupChat` provides several key features:
- Model-based speaker selection
- Configurable participant roles and descriptions
- Optional prevention of consecutive turns by the same speaker
- Customizable selection prompting
- Customizable selection function to override the default model-based selection

```{note}
{py:class}`~autogen_agentchat.teams.SelectorGroupChat` is a high-level API.
If you need more control and customization that is not supported by this API,
you can take a look at the [Group Chat Pattern](../../core-user-guide/design-patterns/group-chat.ipynb)
in the Core API documentation and implement your own group chat logic.
```

## How does it work?

{py:class}`~autogen_agentchat.teams.SelectorGroupChat` is a group chat similar to {py:class}`~autogen_agentchat.teams.RoundRobinGroupChat`,
but with a model-based next speaker selection mechanism.
When the team receives a task through {py:meth}`~autogen_agentchat.teams.BaseGroupChat.run` or {py:meth}`~autogen_agentchat.teams.BaseGroupChat.run_stream`,
the following steps are executed:

1. The team analyzes the current conversation context, including the conversation history and participants' {py:attr}`~autogen_agentchat.base.ChatAgent.name` and {py:attr}`~autogen_agentchat.base.ChatAgent.description` attributes, to determine the next speaker using a model. You can override the model by providing a custom selection function.
2. The team prompts the selected speaker agent to provide a response, which is then **broadcasted** to all other participants.
3. The termination condition is checked to determine if the conversation should end, if not, the process repeats from step 1.
4. When the conversation ends, the team returns the {py:class}`~autogen_agentchat.base.TaskResult` containing the conversation history from this task.

Once the team finishes the task, the conversation context is kept within the team and all participants, so the next task can continue from the previous conversation context.
You can reset the conversation context by calling {py:meth}`~autogen_agentchat.teams.BaseGroupChat.reset`.

In this section, we will demonstrate how to use {py:class}`~autogen_agentchat.teams.SelectorGroupChat` with a simple example for a web search and data analysis task.

## Web Search and Analysis Example

In [1]:
from typing import Sequence

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.messages import AgentMessage
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

from dotenv import load_dotenv
import os

load_dotenv()

api_version = os.getenv("AZURE_OPENAI_API_VERSION")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_chat_completion_client = AzureOpenAIChatCompletionClient(
            model="gpt-4o",
            azure_endpoint=azure_endpoint,
            api_version=api_version,
            api_key=api_key
)

### Agents

![Selector Group Chat](selector-group-chat.svg)

This system uses three specialized agents:

- **Planning Agent**: The strategic coordinator that breaks down complex tasks into manageable subtasks. 
- **Web Search Agent**: An information retrieval specialist that interfaces with the `search_web_tool`.
- **Data Analyst Agent**: An agent specialist in performing calculations equipped with `percentage_change_tool`. 

The tools `search_web_tool` and `percentage_change_tool` are external tools that the agents can use to perform their tasks.

In [2]:
# Note: This example uses mock tools instead of real APIs for demonstration purposes
def search_web_tool(query: str) -> str:
    if "2006-2007" in query:
        return """Here are the total points scored by Miami Heat players in the 2006-2007 season:
        Udonis Haslem: 844 points
        Dwayne Wade: 1397 points
        James Posey: 550 points
        ...
        """
    elif "2007-2008" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2007-2008 is 214."
    elif "2008-2009" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2008-2009 is 398."
    return "No data found."


def percentage_change_tool(start: float, end: float) -> float:
    return ((end - start) / start) * 100

Let's create the specialized agents using the {py:class}`~autogen_agentchat.agents.AssistantAgent` class.
It is important to note that the agents' {py:attr}`~autogen_agentchat.base.ChatAgent.name` and {py:attr}`~autogen_agentchat.base.ChatAgent.description` attributes are used by the model to determine the next speaker,
so it is recommended to provide meaningful names and descriptions.

In [3]:
model_client = azure_openai_chat_completion_client

planning_agent = AssistantAgent(
    "PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        Web search agent: Searches for information
        Data analyst: Performs calculations

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
)

web_search_agent = AssistantAgent(
    "WebSearchAgent",
    description="A web search agent.",
    tools=[search_web_tool],
    model_client=model_client,
    system_message="""
    You are a web search agent.
    Your only tool is search_tool - use it to find information.
    You make only one search call at a time.
    Once you have the results, you never do calculations based on them.
    """,
)

data_analyst_agent = AssistantAgent(
    "DataAnalystAgent",
    description="A data analyst agent. Useful for performing calculations.",
    model_client=model_client,
    tools=[percentage_change_tool],
    system_message="""
    You are a data analyst.
    Given the tasks you have been assigned, you should analyze the data and provide results using the tools provided.
    """,
)

### Workflow

1. The task is received by the {py:class}`~autogen_agentchat.teams.SelectorGroupChat` which, based on agent descriptions, selects the most appropriate agent to handle the initial task (typically the Planning Agent).

2. The **Planning Agent** analyzes the task and breaks it down into subtasks, assigning each to the most appropriate agent using the format:
   `<agent> : <task>`

3. Based on the conversation context and agent descriptions, the {py:class}`~autogen_agent.teams.SelectorGroupChat` manager dynamically selects the next agent to handle their assigned subtask.

4. The **Web Search Agent** performs searches one at a time, storing results in the shared conversation history.

5. The **Data Analyst** processes the gathered information using available calculation tools when selected.

6. The workflow continues with agents being dynamically selected until either:
   - The Planning Agent determines all subtasks are complete and sends "TERMINATE"
   - An alternative termination condition is met (e.g., a maximum number of messages)

When defining your agents, make sure to include a helpful {py:attr}`~autogen_agentchat.base.ChatAgent.description` since this is used to decide which agent to select next.

Let's create the team with two termination conditions:
{py:class}`~autogen_agentchat.conditions.TextMentionTermination` to end the conversation when the Planning Agent sends "TERMINATE",
and {py:class}`~autogen_agentchat.conditions.MaxMessageTermination` to limit the conversation to 25 messages to avoid infinite loop.

In [4]:
text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination

team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent],
    model_client=azure_openai_chat_completion_client,
    termination_condition=termination,
)

Now we run the team with a task to find information about an NBA player.

In [5]:
task = "Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"

# Use asyncio.run(...) if you are running this in a script.
await Console(team.run_stream(task=task))

---------- user ----------
Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?


/home/andy/works/openai/autogen04-sample/.venv/lib/python3.11/site-packages/autogen_agentchat/teams/_group_chat/_selector_group_chat.py:135: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  response = await self._model_client.create(messages=select_speaker_messages)


---------- PlanningAgent ----------
1. Web search agent: Find out who was the Miami Heat player with the highest points in the 2006-2007 season.
2. Web search agent: Find the total rebounds for this player in the 2007-2008 season.
3. Web search agent: Find the total rebounds for this player in the 2008-2009 season.
4. Data analyst: Calculate the percentage change in this player's total rebounds between the 2007-2008 and 2008-2009 seasons.
[Prompt tokens: 159, Completion tokens: 105]


/home/andy/works/openai/autogen04-sample/.venv/lib/python3.11/site-packages/autogen_agentchat/agents/_assistant_agent.py:343: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  result = await self._model_client.create(


---------- WebSearchAgent ----------
[FunctionCall(id='call_DqjIxj6MKqsRP5H9JlBh9j2r', arguments='{"query":"Miami Heat player with the highest points in the 2006-2007 season"}', name='search_web_tool')]
[Prompt tokens: 254, Completion tokens: 30]
---------- WebSearchAgent ----------
[FunctionExecutionResult(content='Here are the total points scored by Miami Heat players in the 2006-2007 season:\n        Udonis Haslem: 844 points\n        Dwayne Wade: 1397 points\n        James Posey: 550 points\n        ...\n        ', call_id='call_DqjIxj6MKqsRP5H9JlBh9j2r')]
---------- WebSearchAgent ----------
Here are the total points scored by Miami Heat players in the 2006-2007 season:
        Udonis Haslem: 844 points
        Dwayne Wade: 1397 points
        James Posey: 550 points
        ...
        
---------- PlanningAgent ----------
1. Web search agent: Confirm that Dwyane Wade was the Miami Heat player with the highest points in the 2006-2007 season.
2. Web search agent: Find the total reb

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=159, completion_tokens=105), content="1. Web search agent: Find out who was the Miami Heat player with the highest points in the 2006-2007 season.\n2. Web search agent: Find the total rebounds for this player in the 2007-2008 season.\n3. Web search agent: Find the total rebounds for this player in the 2008-2009 season.\n4. Data analyst: Calculate the percentage change in this player's total rebounds between the 2007-2008 and 2008-2009 seasons.", type='TextMessage'), ToolCallRequestEvent(source='WebSearchAgent', models_usage=RequestUsage(prompt_tokens=254, completion_tokens=30), content=[FunctionCall(id='call_DqjIxj6MKqsRP5H9JlBh9j2r', a

As we can see, after the Web Search Agent conducts the necessary searches and the Data Analyst Agent completes the necessary calculations, we find that Dwayne Wade was the Miami Heat player with the highest points in the 2006-2007 season, and the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons is 85.98%!

### Custom Selector Function

Often times we want better control over the selection process. 
To this end, we can set the `selector_func` argument with a custom selector function to override the default model-based selection.
For instance, we want the Planning Agent to speak immediately after any specialized agent to check the progress.

```{note}
Returning `None` from the custom selector function will use the default model-based selection.
```

In [6]:
def selector_func(messages: Sequence[AgentMessage]) -> str | None:
    if messages[-1].source != planning_agent.name:
        return planning_agent.name
    return None


# Reset the previous team and run the chat again with the selector function.
# await team.reset()
team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent],
    model_client=azure_openai_chat_completion_client,
    termination_condition=termination,
    selector_func=selector_func,
)

await Console(team.run_stream(task=task))

---------- user ----------
Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?


---------- PlanningAgent ----------
The Miami Heat player with the highest points in the 2006-2007 season was Dwyane Wade, who scored 1397 points. The total rebounds for Dwyane Wade in the 2007-2008 season were 214, and in the 2008-2009 season, they were 398. The percentage change in his total rebounds between these two seasons is approximately 85.98%.

TERMINATE
[Prompt tokens: 712, Completion tokens: 88]
---------- Summary ----------
Number of messages: 2
Finish reason: Text 'TERMINATE' mentioned
Total prompt tokens: 712
Total completion tokens: 88
Duration: 2.51 seconds


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=712, completion_tokens=88), content='The Miami Heat player with the highest points in the 2006-2007 season was Dwyane Wade, who scored 1397 points. The total rebounds for Dwyane Wade in the 2007-2008 season were 214, and in the 2008-2009 season, they were 398. The percentage change in his total rebounds between these two seasons is approximately 85.98%.\n\nTERMINATE', type='TextMessage')], stop_reason="Text 'TERMINATE' mentioned")

You can see from the conversation log that the Planning Agent always speaks immediately after the specialized agents.